# Taller 1 - Red de Monitoreo de Calidad del Aire de Bogota.
http://rmcab.ambientebogota.gov.co/Report/stationreport

## Decripción del conjunto de datos

# 1. Instalar Librerias

In [1]:
!pip install --upgrade pip # se actualiza pip, en caso de que no lo tenga actualizado
!pip install psycopg2-binary # Psycopg es un adaptador de base de datos PostgreSQL
!pip install pandas 
!pip install openpyxl


  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.3
    Uninstalling pip-23.3:
      Successfully uninstalled pip-23.3
  Using cached psycopg2_binary-2.9.11-cp311-cp311-manylinux2014_aarch64.manylinux_2_17_aarch64.whl.metadata (4.9 kB)
Using cached psycopg2_binary-2.9.11-cp311-cp311-manylinux2014_aarch64.manylinux_2_17_aarch64.whl (4.4 MB)


## 1.1 Importar librerias

In [2]:
import pandas as pd
from os import walk
import os, re   #expresiones regulares

# 2. Procesar conjunto de datos (Preprocesssing-dataset)

## 2.1 Unir todos los conjuntos de datos de extensión xlsx en un arreglo 


In [3]:
# unir todos los conjuntos de datos de extensión xlsx en un arreglo 
arregloDeDataSets = []
for (dirpath, dirnames, filenames) in walk('data/raw/'):
    arregloDeDataSets.extend(filenames)
    break
arregloDeDataSets

['ca_USM_StationsReport_202231291350.xlsx',
 'ca_BOL_StationsReport_20223129119.xlsx',
 'ca_SUB_StationsReport_202231291211.xlsx',
 'ca_TUN_StationsReport_202231291241.xlsx',
 'ca_LFR_StationsReport_20223129750.xlsx',
 'ca_PTE_StationsReport_20223129110.xlsx',
 'ca_MAM_StationsReport_20223129831.xlsx',
 'ca_CBV_StationsReport_20223129333.xlsx',
 'ca_CDAR_StationsReport_20223129247.xlsx',
 'ca_SCR_StationsReport_202231291142.xlsx',
 'ca_FTB_StationsReport_20223129456.xlsx',
 'ca_JAZ_StationsReport_20223129614.xlsx',
 'ca_7MA_StationsReport_20223129920.xlsx',
 'ca_CSE_StationsReport_20223129152.xlsx',
 'ca_GYR_StationsReport_20223129531.xlsx',
 'ca_COL_StationsReport_20223129424.xlsx',
 'ca_MOV2_StationsReport_202231291026.xlsx',
 'ca_USQ_StationsReport_202231291319.xlsx',
 'ca_KEN_StationsReport_2022312979.xlsx']

## 2.2  Concatenar los datsets en uno solo 


In [4]:
path = os.path.abspath('data/raw/') 
files = os.listdir(path)  

df = pd.DataFrame()
for file in files:
    if file.endswith('.xlsx'):
        dft = pd.read_excel('data/raw/'+file)
        dft['Station'] = re.search(r'_(.*?)_', str(file)).group(1)
        df = pd.concat([df, dft[1:]], ignore_index=True)
df.head() 

,DateTime,PM10,PM2.5,NO,NO2,NOX,SO2,CO,OZONO,Vel Viento,...,HR.1,Canal no activo,PM10 Flow,CO2,SO2 Envea,Vel Viento 10M,Dir Viento 10M,Temperatura 8M,Temperatura 20M,PM2.5 Flow
0,01-01-2021 01:00,56.6,32.7,7.504,15.962,23.493,7.24,0.44924,2.431,0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01-01-2021 02:00,59.3,39.3,16.56,17.866,34.426,5.405,0.69832,1.121,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01-01-2021 03:00,96.4,70.8,22.989,17.802,40.791,5.685,0.88243,1.172,0.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01-01-2021 04:00,108.3,81,3.704,9.886,13.591,2.436,0.29549,6.565,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01-01-2021 05:00,87.7,56.1,2.098,9.272,11.371,3.039,0.16621,9.513,0.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.3 Comprobar los valores nulos por columnas

In [5]:
#Contar cuantos valores nulos hay por cada columna
df.isnull().sum()

DateTime                0
PM10                    0
PM2.5                   0
NO                   8760
NO2                  8760
NOX                  8760
SO2                 43800
CO                      0
OZONO                8760
Vel Viento          26280
Dir Viento          26280
Temperatura         17520
HR                  52560
Presion Baro        70080
Rad Solar           61320
Station                 0
Precipitacion       61320
HR.1               157680
Canal no activo    157680
PM10 Flow          157680
CO2                148920
SO2 Envea          157680
Vel Viento 10M     157680
Dir Viento 10M     157680
Temperatura 8M     157680
Temperatura 20M    157680
PM2.5 Flow         157680
dtype: int64

## 2.4 Organizar y guardar en un nuevo df las columnas significativas

In [6]:
#Organizar y guardar en un nuevo df las columnas significativas
df = df[['PM10','PM2.5','NO','NO2','NOX','CO','OZONO','Station', 'DateTime']]
df.head()

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime
0,56.6,32.7,7.504,15.962,23.493,0.44924,2.431,USM,01-01-2021 01:00
1,59.3,39.3,16.56,17.866,34.426,0.69832,1.121,USM,01-01-2021 02:00
2,96.4,70.8,22.989,17.802,40.791,0.88243,1.172,USM,01-01-2021 03:00
3,108.3,81,3.704,9.886,13.591,0.29549,6.565,USM,01-01-2021 04:00
4,87.7,56.1,2.098,9.272,11.371,0.16621,9.513,USM,01-01-2021 05:00


## 2.5 Convertir valores a numericos y sino poner NaN

In [7]:
#Aquí su código 
# Lista de columnas que deberían ser numéricas
cols_numericas = ['PM10','PM2.5','NO','NO2','NOX','CO','OZONO']
df[cols_numericas] = df[cols_numericas].apply(pd.to_numeric, errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166440 entries, 0 to 166439
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   PM10      146426 non-null  float64
 1   PM2.5     151128 non-null  float64
 2   NO        138776 non-null  float64
 3   NO2       138778 non-null  float64
 4   NOX       138772 non-null  float64
 5   CO        135202 non-null  float64
 6   OZONO     134308 non-null  float64
 7   Station   166440 non-null  object 
 8   DateTime  166440 non-null  object 
dtypes: float64(7), object(2)
memory usage: 11.4+ MB


In [8]:
df.isnull().sum()

PM10        20014
PM2.5       15312
NO          27664
NO2         27662
NOX         27668
CO          31238
OZONO       32132
Station         0
DateTime        0
dtype: int64

## 2.6 Ver el tamaño del dataset 

In [9]:
#ver el tamaño del conjunto de datos 
#Aquí su código 
df.shape

(166440, 9)

## 2.7 Crear una nueva columna con filtro (False o True)

In [10]:
# True is greater than 12, bad (Pure, Not Pure)
df['Status'] = df['PM2.5']>12
df.Status.value_counts()
df.head()

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime,Status
0,56.6,32.7,7.504,15.962,23.493,0.44924,2.431,USM,01-01-2021 01:00,True
1,59.3,39.3,16.560,17.866,34.426,0.69832,1.121,USM,01-01-2021 02:00,True
2,96.4,70.8,22.989,17.802,40.791,0.88243,1.172,USM,01-01-2021 03:00,True
3,108.3,81.0,3.704,9.886,13.591,0.29549,6.565,USM,01-01-2021 04:00,True
4,87.7,56.1,2.098,9.272,11.371,0.16621,9.513,USM,01-01-2021 05:00,True


## 2.8 Guardar todo el proceso en un nuevo dataset

In [11]:
# we need another column to specify stations
#df.to_csv("data/clean_data_final.csv", index = False)
#Aquí su código 

df.to_csv("data/clean_data_final.csv", index = False)

# 3. Integrar nuevo dataset (Stations)

## 3.1 Cargar dataset de stations

In [12]:
#cargar el df dataset_with_missing.csv   data/dataset_with_missing.csv'
#cargar el df stations_loc.csv data/stations_loc.csv')

#Aquí su código 
missing = pd.read_csv('data/dataset_with_missing.csv')
stations = pd.read_csv('data/stations_loc.csv')

### 3.1.1 Seleccionar algunas columnas del dataset stations

In [13]:
stations = stations[['Sigla', 'Latitud', 'Longitud', "estacion", "Localidad"]] 


### 3.1.2 Cambiar el nombre de una columna 

In [14]:
stations = stations.rename(columns={'Sigla': 'Station'}) #Columna Sigla por Station
stations.head()

,Station,Latitud,Longitud,estacion,Localidad
0,GYR,"4°47'01.5""N","74°02'38.9""W",guaymaral,Suba
1,USQ,"4°42'37.26""N","74°1'49.50""W",usaquen,Usaquén
2,SUB,"4°45'40.49""N","74° 5'36.46""W",suba,Suba
3,BOL,"4°44'08.9""N","74°07'33.2""W",bolivia,Engativá
4,LFR,"4°41'26.52""N","74°4'56.94""W",las_ferias,Engativá


## 3.2 Convertir las coordenadas de texto a decimales

In [15]:
import re

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'S' or direction == 'W':
        dd *= -1
    return dd;

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

def parse_dms(coor):
    parts = re.split('[^\d\w]+', coor)
    dec_coor = dms2dd(parts[0], parts[1], float(parts[2]+'.'+parts[2]), parts[4])
    return dec_coor

'''
def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'E' or direction == 'S':
        dd *= -1
    return dd;

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

def parse_dms(dms):
    parts = re.split('[^\d\w]+', dms)
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])
 
    return (lat)
'''

"\ndef dms2dd(degrees, minutes, seconds, direction):\n    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);\n    if direction == 'E' or direction == 'S':\n        dd *= -1\n    return dd;\n\ndef dd2dms(deg):\n    d = int(deg)\n    md = abs(deg - d) * 60\n    m = int(md)\n    sd = (md - m) * 60\n    return [d, m, sd]\n\ndef parse_dms(dms):\n    parts = re.split('[^\\d\\w]+', dms)\n    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])\n \n    return (lat)\n"

In [16]:
#Aplicar la funcion parse_dms a la columna Latitud y Longitud
stations['Latitud'] = stations['Latitud'].apply(parse_dms)
stations['Longitud'] = stations['Longitud'].apply(parse_dms)
stations.head()

,Station,Latitud,Longitud,estacion,Localidad
0,GYR,4.783614,-74.043994,guaymaral,Suba
1,USQ,4.710381,-74.030414,usaquen,Usaquén
2,SUB,4.761222,-74.093433,suba,Suba
3,BOL,4.735578,-74.125925,bolivia,Engativá
4,LFR,4.690628,-74.082378,las_ferias,Engativá


## 3.3 Integrar el dataset df con el dataset stations

In [17]:
df = pd.merge(df, stations, on='Station', how='inner')
df.head(3)

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime,Status,Latitud,Longitud,estacion,Localidad
0,56.6,32.7,7.504,15.962,23.493,0.44924,2.431,USM,01-01-2021 01:00,True,4.532097,-74.116947,usame,Usme
1,59.3,39.3,16.560,17.866,34.426,0.69832,1.121,USM,01-01-2021 02:00,True,4.532097,-74.116947,usame,Usme
2,96.4,70.8,22.989,17.802,40.791,0.88243,1.172,USM,01-01-2021 03:00,True,4.532097,-74.116947,usame,Usme


## 3.4 Convertir hora (DateTime)

In [18]:
#Puede ver que en la columna 'DateTime', 
#la información sobre la fecha y la hora se dan juntas. 
#Por lo tanto, extraerá la información de tiempo.

def replace24(datetimex):
    return datetimex.replace('24:00', '00:00') #cambia de horarios

In [19]:
# Esta celda extraerá información de la columna 'datetime' y 
#generará columnas de meses, días o semanas y horas
df['DateTime'] = df['DateTime'].apply(replace24)
df['DateTime'] = pd.to_datetime(df['DateTime'], dayfirst=True)
df['month'] = pd.DatetimeIndex(df['DateTime']).month
df['day_week'] = pd.DatetimeIndex(df['DateTime']).weekday
df['day_month'] = pd.DatetimeIndex(df['DateTime']).day
df['hour'] = pd.DatetimeIndex(df['DateTime']).hour
df.loc[df['hour']==0,'hour'] = 24

In [20]:
df.head()

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime,Status,Latitud,Longitud,estacion,Localidad,month,day_week,day_month,hour
0,56.6,32.7,7.504,15.962,23.493,0.44924,2.431,USM,2021-01-01 01:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,1
1,59.3,39.3,16.560,17.866,34.426,0.69832,1.121,USM,2021-01-01 02:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,2
2,96.4,70.8,22.989,17.802,40.791,0.88243,1.172,USM,2021-01-01 03:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,3
3,108.3,81.0,3.704,9.886,13.591,0.29549,6.565,USM,2021-01-01 04:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,4
4,87.7,56.1,2.098,9.272,11.371,0.16621,9.513,USM,2021-01-01 05:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,5


## 3.5 Guardar todo el proceso en un nuevo dataset

In [21]:
df.rename(columns={"estacion": "Name"}, inplace=True)
df.to_csv("data/dataset_with_geo_missing.csv", index = False)

# 4. Limpieza de datos

In [22]:
# Varias columnas tienen valores perdidos en el dataset

#Aquí su código 
df.isnull().sum()

PM10         20014
PM2.5        15312
NO           27664
NO2          27662
NOX          27668
CO           31238
OZONO        32132
Station          0
DateTime         0
Status           0
Latitud          0
Longitud         0
Name             0
Localidad        0
month            0
day_week         0
day_month        0
hour             0
dtype: int64

## 4.1 Remplazar los valores NaN en cada columna con su media

In [23]:
df['PM10'].fillna((df['PM10'].mean()), inplace=True)

#hagalo para PM2.5, CO, NO, NO2, NOX, OZONO
#Aquí su código
df['PM2.5'].fillna((df['PM2.5'].mean()), inplace=True)
df['CO'].fillna((df['CO'].mean()), inplace=True)
df['NO'].fillna((df['NO'].mean()), inplace=True)
df['NO2'].fillna((df['NO2'].mean()), inplace=True)
df['NOX'].fillna((df['NOX'].mean()), inplace=True)
df['OZONO'].fillna((df['OZONO'].mean()), inplace=True)

In [24]:
# compruebe --- ya no hay valores perdidos en el dataset

df.isnull().sum()

PM10         0
PM2.5        0
NO           0
NO2          0
NOX          0
CO           0
OZONO        0
Station      0
DateTime     0
Status       0
Latitud      0
Longitud     0
Name         0
Localidad    0
month        0
day_week     0
day_month    0
hour         0
dtype: int64

## 4.2 Guardar todo el proceso en un nuevo dataset

In [25]:
df.to_csv("data/dataset_final_clean_mean.csv", index = False)

# 5. Creando el esquema de la bodega de datos 

In [26]:
df = pd.read_csv('data/dataset_final_clean_mean.csv')
df.head(10)

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,Station,DateTime,Status,Latitud,Longitud,Name,Localidad,month,day_week,day_month,hour
0,56.6,32.7,7.504,15.962,23.493,0.449240,2.431,USM,2021-01-01 01:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,1
1,59.3,39.3,16.560,17.866,34.426,0.698320,1.121,USM,2021-01-01 02:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,2
2,96.4,70.8,22.989,17.802,40.791,0.882430,1.172,USM,2021-01-01 03:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,3
3,108.3,81.0,3.704,9.886,13.591,0.295490,6.565,USM,2021-01-01 04:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,4
4,87.7,56.1,2.098,9.272,11.371,0.166210,9.513,USM,2021-01-01 05:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,5
5,74.4,38.6,2.249,11.064,13.313,0.225340,5.466,USM,2021-01-01 06:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,6
6,60.4,27.2,11.490,14.675,26.165,0.413570,2.467,USM,2021-01-01 07:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,7
7,51.9,29.5,5.864,7.348,13.212,0.207840,10.800,USM,2021-01-01 08:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,8
8,38.1,21.6,3.506,4.266,7.773,0.691233,17.161,USM,2021-01-01 09:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,9
9,28.9,23.1,4.343,3.866,8.209,0.691233,19.674,USM,2021-01-01 10:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,10


## 5.1 Se crea un nuevo DataFrame con los datos de los polutantes

In [27]:
df_polutante=pd.DataFrame(df, columns=["PM10", "PM2.5", "NO", "NO2", "NOX", "CO", "OZONO"]) # se crea un nuevo DataFrame unicamente con los polutantes
df['id_polutante'] = df.index+1 # se crea una nueva columna que identificará a cada polutante
df_polutante

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO
0,56.6,32.7,7.504,15.962,23.493,0.44924,2.431
1,59.3,39.3,16.560,17.866,34.426,0.69832,1.121
2,96.4,70.8,22.989,17.802,40.791,0.88243,1.172
3,108.3,81.0,3.704,9.886,13.591,0.29549,6.565
4,87.7,56.1,2.098,9.272,11.371,0.16621,9.513
...,...,...,...,...,...,...,...
166435,43.1,12.0,11.771,23.697,35.468,0.98779,8.146
166436,35.0,35.0,6.634,21.677,28.311,0.85772,8.055
166437,43.5,43.0,2.666,19.867,22.533,0.84339,6.877
166438,60.1,55.0,2.622,19.689,22.311,0.84038,6.533


## 5.2 Se crea un nuevo dataset con los datos de las estaciones y se eliminan las filas repetidas

In [28]:
df_estacion=pd.DataFrame(df, columns=["Name", "Station", "Localidad", "Latitud", "Longitud"]) # se crea un nuevo DataFrame unicamente con los datos de las estaciones
df_estacion = df_estacion.drop_duplicates() # se eliminan las filas repetidas
df_estacion = df_estacion.rename(columns={'Station': 'Sigla'}) # se cambia el nombre a la columna Station por Sigla
df_estacion

,Name,Sigla,Localidad,Latitud,Longitud
0,usame,USM,Usme,4.532097,-74.116947
8760,bolivia,BOL,Engativá,4.735578,-74.125925
17520,suba,SUB,Suba,4.761222,-74.093433
26280,tunal,TUN,Tunjuelito,4.576206,-74.130975
35040,las_ferias,LFR,Engativá,4.690628,-74.082378
43800,pueste_aranda,PTE,Puente Aranda,4.631817,-74.117278
52560,minambiente,MAM,Santa Fe,4.625364,-74.066972
61320,ciudad_bolivar,CBV,Ciudad Bolívar,4.577889,-74.166272
70080,centro_alto_rendimiento,CDAR,Engativá,4.658417,-74.083944
78840,san_cristobal,SCR,San Cristóbal,4.572558,-74.083639


In [29]:
# se remplaza la columna sigla, dejando unicamente los datos unicos y luego se le cambia el nombre
# a la misma por id_estacion
count = 1
for index, row in df_estacion.iterrows():
    df = df.replace({row["Sigla"]: count})
    count += 1
df = df.rename(columns={'Station': 'id_estacion'})
df

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,id_estacion,DateTime,Status,Latitud,Longitud,Name,Localidad,month,day_week,day_month,hour,id_polutante
0,56.6,32.7,7.504,15.962,23.493,0.44924,2.431,1,2021-01-01 01:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,1,1
1,59.3,39.3,16.560,17.866,34.426,0.69832,1.121,1,2021-01-01 02:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,2,2
2,96.4,70.8,22.989,17.802,40.791,0.88243,1.172,1,2021-01-01 03:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,3,3
3,108.3,81.0,3.704,9.886,13.591,0.29549,6.565,1,2021-01-01 04:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,4,4
4,87.7,56.1,2.098,9.272,11.371,0.16621,9.513,1,2021-01-01 05:00:00,True,4.532097,-74.116947,usame,Usme,1,4,1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166435,43.1,12.0,11.771,23.697,35.468,0.98779,8.146,19,2021-12-31 20:00:00,False,4.625083,-74.161222,kennedy,Kennedy,12,4,31,20,166436
166436,35.0,35.0,6.634,21.677,28.311,0.85772,8.055,19,2021-12-31 21:00:00,True,4.625083,-74.161222,kennedy,Kennedy,12,4,31,21,166437
166437,43.5,43.0,2.666,19.867,22.533,0.84339,6.877,19,2021-12-31 22:00:00,True,4.625083,-74.161222,kennedy,Kennedy,12,4,31,22,166438
166438,60.1,55.0,2.622,19.689,22.311,0.84038,6.533,19,2021-12-31 23:00:00,True,4.625083,-74.161222,kennedy,Kennedy,12,4,31,23,166439


## 5.3 Se crea un nuevo DataFrame con los datos de las fechas.

In [30]:
# se crea un nuevo DataFrame y se eliminan los valores repetidos
df_fecha=pd.DataFrame(df, columns=["DateTime"]) # se crea un nuevo DataFrame con las fechas y horas 
#df_fecha["DateTime"].value_counts() # !TODO Borrar?
df_fecha = df_fecha.drop_duplicates() # se eliminan las filas repetidas
df_fecha

,DateTime
0,2021-01-01 01:00:00
1,2021-01-01 02:00:00
2,2021-01-01 03:00:00
3,2021-01-01 04:00:00
4,2021-01-01 05:00:00
...,...
8755,2021-12-31 20:00:00
8756,2021-12-31 21:00:00
8757,2021-12-31 22:00:00
8758,2021-12-31 23:00:00


In [31]:
import re # libreria para el uso de expresiones regulares
from pandas.tseries.holiday import Holiday, AbstractHolidayCalendar
from datetime import datetime

# definimos los días festivo para el año en que se tomaron los datos
class EsBusinessCalendar(AbstractHolidayCalendar):
   rules = [ # se definen los festivos, teniendo en cuenta los horarios festivos para el año de 2021 en Colombia
     Holiday('Año Nuevo', month=1, day=1),
     Holiday('Día de los Reyes Magos', month=1, day=11),
     Holiday('Día de San José', month=3, day=22),
     Holiday('Jueves Santo', month=4, day=1),
     Holiday('Viernes Santo', month=4, day=2),
     Holiday('Día del Trabajador', month=5, day=1),
     Holiday('Día de la Ascensión', month=5, day=17),
     Holiday('Corpus Christi', month=6, day=7),
     Holiday('Sagrado Corazón', month=6, day=14),
     Holiday('San Pedro y San Pablo ', month=7, day=5),
     Holiday('Día de la Independencia', month=7, day=20),
     Holiday('Batalla de Boyacá', month=8, day=7),
     Holiday('Asunción de la Virgen', month=8, day=16),
     Holiday('Celebración del Día de la Raza', month=10, day=18),
     Holiday('Día de todos los Santos', month=11, day=1),
     Holiday('Independencia de Cartagena', month=11, day=15),
     Holiday('Inmaculada Concepción', month=12, day=8),    
     Holiday('Navidad', month=12, day=25)
   ]

calendar_festivos = EsBusinessCalendar() # se instancia el objeto
calendar_festivos = calendar_festivos.holidays(start='2021-01-01', end='2021-12-31') # se define el rango de tiempo en que se tendran en cuenta los festivos (anho 2021)

dias = []
meses = []
anhos = []
horas = []
fin_semana = []
festivo = []


for index, row in df_fecha.iterrows(): # se recorre cada fila del DataFrame
    list_fecha = re.split("[\-\s]", row['DateTime']) # 
    dias.append(list_fecha[2])
    meses.append(list_fecha[1])
    anhos.append(list_fecha[0])
    horas.append(list_fecha[3])

    hora = row['DateTime'].split(" ")
    hora[0] = datetime.strptime(hora[0], '%Y-%m-%d')
    if hora[0].weekday() < 5 : # condicción para determinar si el día está comprendido entre lues-viernes
        fin_semana.append(False)
    else:
        fin_semana.append(True)
    if hora[0] in calendar_festivos: # condicción para determinar si el día hace parte de los días festivos del año
        festivo.append(True)
    else: 
        festivo.append(False)

dict_fechas = { 'dia': dias, 'mes': meses, 'anho': anhos, 'hora': horas, 'fin_semana': fin_semana, 'festivo': festivo}
df_fechas = pd.DataFrame(data=dict_fechas)
df_fechas


,dia,mes,anho,hora,fin_semana,festivo
0,01,01,2021,01:00:00,False,True
1,01,01,2021,02:00:00,False,True
2,01,01,2021,03:00:00,False,True
3,01,01,2021,04:00:00,False,True
4,01,01,2021,05:00:00,False,True
...,...,...,...,...,...,...
8755,31,12,2021,20:00:00,False,False
8756,31,12,2021,21:00:00,False,False
8757,31,12,2021,22:00:00,False,False
8758,31,12,2021,23:00:00,False,False


## 5.4 Creación de la columna id_tiempo para identificar cada una de las fechas en el DF principal

In [32]:
count = 1
# se recorre cada fecha de df_fecha
for index, row in df_fecha.iterrows(): 
# por cada fila de df_fecha se crea un nuevo identificador en el DataFrame principal
    df.loc[df['DateTime'] == row["DateTime"], 'DateTime'] = count     
    count += 1
df = df.rename(columns={'DateTime': 'id_tiempo'})
df

,PM10,PM2.5,NO,NO2,NOX,CO,OZONO,id_estacion,id_tiempo,Status,Latitud,Longitud,Name,Localidad,month,day_week,day_month,hour,id_polutante
0,56.6,32.7,7.504,15.962,23.493,0.44924,2.431,1,1,True,4.532097,-74.116947,usame,Usme,1,4,1,1,1
1,59.3,39.3,16.560,17.866,34.426,0.69832,1.121,1,2,True,4.532097,-74.116947,usame,Usme,1,4,1,2,2
2,96.4,70.8,22.989,17.802,40.791,0.88243,1.172,1,3,True,4.532097,-74.116947,usame,Usme,1,4,1,3,3
3,108.3,81.0,3.704,9.886,13.591,0.29549,6.565,1,4,True,4.532097,-74.116947,usame,Usme,1,4,1,4,4
4,87.7,56.1,2.098,9.272,11.371,0.16621,9.513,1,5,True,4.532097,-74.116947,usame,Usme,1,4,1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166435,43.1,12.0,11.771,23.697,35.468,0.98779,8.146,19,8756,False,4.625083,-74.161222,kennedy,Kennedy,12,4,31,20,166436
166436,35.0,35.0,6.634,21.677,28.311,0.85772,8.055,19,8757,True,4.625083,-74.161222,kennedy,Kennedy,12,4,31,21,166437
166437,43.5,43.0,2.666,19.867,22.533,0.84339,6.877,19,8758,True,4.625083,-74.161222,kennedy,Kennedy,12,4,31,22,166438
166438,60.1,55.0,2.622,19.689,22.311,0.84038,6.533,19,8759,True,4.625083,-74.161222,kennedy,Kennedy,12,4,31,23,166439


In [33]:
# finalmente se crea un DataFrame, el cual se relaciona con los DataFrame creados anteriormente por medio de sus identificadores 
df_fact_medidad=pd.DataFrame(df, columns=["id_estacion", "id_tiempo", "id_polutante"])
df_fact_medidad

,id_estacion,id_tiempo,id_polutante
0,1,1,1
1,1,2,2
2,1,3,3
3,1,4,4
4,1,5,5
...,...,...,...
166435,19,8756,166436
166436,19,8757,166437
166437,19,8758,166438
166438,19,8759,166439


## 5.6Creación de las tablas en Postgres

In [34]:
# Es necesario tener instaladas las librerias 
!pip install python-dotenv
!pip install psycopg2-binary

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)


In [35]:
from Taller1.conexion import new_model

# se llama a la función new_model (archivo conexion.py) con cada uno de los DataFrame para crear las tablas correspondientes
df_fact_medidad = df_fact_medidad.astype(int)
new_model(df_estacion, "dim_estacion")

#Cree las tablas para dim_fecha, dim_polutante, fact_medidad
new_model(df_fecha, "dim_fecha")
new_model(df_polutante, "dim_polutante")
new_model(df_fact_medidad, "fact_medidad")
